In [1]:
# NOTE: Don't run this Jupyter Notebook! It won't work! It was run on Google Colab.
#       Look at the output of the cells. For demonstration purposes

import cv2
import dlib
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from scipy.spatial import distance as dist
from imutils import face_utils

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip "/content/drive/MyDrive/Feature1_datacrop.zip" -d "/content"

Archive:  /content/drive/MyDrive/Feature1_datacrop.zip
  inflating: /content/Feature1_datacrop/aaaaaaaaaa0.png  
  inflating: /content/Feature1_datacrop/aaaaaaaaaa1.png  
  inflating: /content/Feature1_datacrop/aaaaaaaaaa10.png  
  inflating: /content/Feature1_datacrop/aaaaaaaaaa100.png  
  inflating: /content/Feature1_datacrop/aaaaaaaaaa101.png  
  inflating: /content/Feature1_datacrop/aaaaaaaaaa102.png  
  inflating: /content/Feature1_datacrop/aaaaaaaaaa103.png  
  inflating: /content/Feature1_datacrop/aaaaaaaaaa104.png  
  inflating: /content/Feature1_datacrop/aaaaaaaaaa105.png  
  inflating: /content/Feature1_datacrop/aaaaaaaaaa106.png  
  inflating: /content/Feature1_datacrop/aaaaaaaaaa107.png  
  inflating: /content/Feature1_datacrop/aaaaaaaaaa108.png  
  inflating: /content/Feature1_datacrop/aaaaaaaaaa109.png  
  inflating: /content/Feature1_datacrop/aaaaaaaaaa11.png  
  inflating: /content/Feature1_datacrop/aaaaaaaaaa110.png  
  inflating: /content/Feature1_datacrop/aaaaaaaaaa1

In [4]:
!unzip "/content/drive/MyDrive/Feature1_datatestcrop.zip" -d "/content"

Archive:  /content/drive/MyDrive/Feature1_datatestcrop.zip
  inflating: /content/Feature1_datatestcrop/Realtest0.png  
  inflating: /content/Feature1_datatestcrop/Realtest1.png  
  inflating: /content/Feature1_datatestcrop/Realtest10.png  
  inflating: /content/Feature1_datatestcrop/Realtest100.png  
  inflating: /content/Feature1_datatestcrop/Realtest101.png  
  inflating: /content/Feature1_datatestcrop/Realtest102.png  
  inflating: /content/Feature1_datatestcrop/Realtest103.png  
  inflating: /content/Feature1_datatestcrop/Realtest104.png  
  inflating: /content/Feature1_datatestcrop/Realtest105.png  
  inflating: /content/Feature1_datatestcrop/Realtest106.png  
  inflating: /content/Feature1_datatestcrop/Realtest107.png  
  inflating: /content/Feature1_datatestcrop/Realtest108.png  
  inflating: /content/Feature1_datatestcrop/Realtest109.png  
  inflating: /content/Feature1_datatestcrop/Realtest11.png  
  inflating: /content/Feature1_datatestcrop/Realtest110.png  
  inflating: /con

In [5]:
!unzip "/content/drive/MyDrive/Feature1_datatest2crop.zip" -d "/content"

Archive:  /content/drive/MyDrive/Feature1_datatest2crop.zip
  inflating: /content/Feature1_datatest2crop/usethis0.png  
  inflating: /content/Feature1_datatest2crop/usethis1.png  
  inflating: /content/Feature1_datatest2crop/usethis10.png  
  inflating: /content/Feature1_datatest2crop/usethis100.png  
  inflating: /content/Feature1_datatest2crop/usethis101.png  
  inflating: /content/Feature1_datatest2crop/usethis102.png  
  inflating: /content/Feature1_datatest2crop/usethis103.png  
  inflating: /content/Feature1_datatest2crop/usethis104.png  
  inflating: /content/Feature1_datatest2crop/usethis105.png  
  inflating: /content/Feature1_datatest2crop/usethis106.png  
  inflating: /content/Feature1_datatest2crop/usethis107.png  
  inflating: /content/Feature1_datatest2crop/usethis108.png  
  inflating: /content/Feature1_datatest2crop/usethis109.png  
  inflating: /content/Feature1_datatest2crop/usethis11.png  
  inflating: /content/Feature1_datatest2crop/usethis110.png  
  inflating: /co

In [6]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Oct 29 03:13:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
K.clear_session()

In [8]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True

In [9]:
# Use the code from Chukwudi's notebook... save the plt.show() from the testing set as images for this notebook training set
# Eyes: 0 is closed, 1 is open.
# It dataset is ~65% open, 35% closed.
eyes_df = pd.read_csv('drive/MyDrive/Feature1_labelsmiss.csv', header=0)
eyes_df.groupby('Eyes')['Eyes'].count()

Eyes
0     723
1    1357
Name: Eyes, dtype: int64

In [10]:
eyes_df

,VersionInfo,BaseName,Name,Length,FullName,Eyes
0,File: C:\Users\Michael\Desktop\Mic...,aaaaaaaaaa0,aaaaaaaaaa0.png,60894,C:\Users\Michael\Desktop\Michael All\VS_Projec...,1
1,File: C:\Users\Michael\Desktop\Mic...,aaaaaaaaaa1,aaaaaaaaaa1.png,60992,C:\Users\Michael\Desktop\Michael All\VS_Projec...,1
2,File: C:\Users\Michael\Desktop\Mic...,aaaaaaaaaa10,aaaaaaaaaa10.png,64059,C:\Users\Michael\Desktop\Michael All\VS_Projec...,1
3,File: C:\Users\Michael\Desktop\Mic...,aaaaaaaaaa100,aaaaaaaaaa100.png,58726,C:\Users\Michael\Desktop\Michael All\VS_Projec...,0
4,File: C:\Users\Michael\Desktop\Mic...,aaaaaaaaaa101,aaaaaaaaaa101.png,58709,C:\Users\Michael\Desktop\Michael All\VS_Projec...,0
...,...,...,...,...,...,...
2075,File: C:\Users\Michael\Desktop\Mic...,pxjkzvqomp95,pxjkzvqomp95.png,58975,C:\Users\Michael\Desktop\Michael All\VS_Projec...,1
2076,File: C:\Users\Michael\Desktop\Mic...,pxjkzvqomp96,pxjkzvqomp96.png,54257,C:\Users\Michael\Desktop\Michael All\VS_Projec...,1
2077,File: C:\Users\Michael\Desktop\Mic...,pxjkzvqomp97,pxjkzvqomp97.png,58539,C:\Users\Michael\Desktop\Michael All\VS_Projec...,1
2078,File: C:\Users\Michael\Desktop\Mic...,pxjkzvqomp98,pxjkzvqomp98.png,59206,C:\Users\Michael\Desktop\Michael All\VS_Projec...,1


In [11]:
# Train test split.
X_train, X_test, y_train, y_test = train_test_split(
 eyes_df['Name'],
 eyes_df['Eyes'],
 test_size=0.2,
 random_state=180,
 shuffle=True)
y_test.groupby(y_train).count()

Series([], Name: Eyes, dtype: int64)

In [12]:
print(X_train.iloc[0])
print(len(X_train))

dpevefkefv72.png
1664


In [13]:
X_train

1011     dpevefkefv72.png
896     dpevefkefv102.png
1578     novarhxpbj49.png
215      aayfryxljh23.png
29      aaaaaaaaaa124.png
              ...        
855      dddddddddd67.png
771     dddddddddd125.png
1767     oojxonbgow84.png
2077     pxjkzvqomp97.png
1035     dpevefkefv94.png
Name: Name, Length: 1664, dtype: object

In [14]:
for i in X_test:
    print(i)

opvqdabdap56.png
bbbbbbbbbb73.png
jzmzdispyo64.png
jzmzdispyo136.png
jzmzdispyo62.png
pxjkzvqomp19.png
mkmgcxaztt43.png
eeeeeeeeee48.png
jzmzdispyo143.png
jzmzdispyo96.png
dddddddddd37.png
opvqdabdap67.png
mkmgcxaztt61.png
novarhxpbj109.png
eeeeeeeeee14.png
aaaaaaaaaa72.png
mkmgcxaztt58.png
mkmgcxaztt2.png
jzmzdispyo61.png
aaaaaaaaaa50.png
pxjkzvqomp143.png
jzmzdispyo82.png
mkmgcxaztt49.png
aayfryxljh14.png
opvqdabdap13.png
dddddddddd31.png
aayfryxljh81.png
novarhxpbj101.png
bbbbbbbbbb145.png
pxjkzvqomp58.png
jzmzdispyo75.png
jzmzdispyo33.png
pxjkzvqomp88.png
oojxonbgow60.png
adohdulfwb62.png
aaaaaaaaaa110.png
mkmgcxaztt135.png
bbbbbbbbbb87.png
cccccccccc97.png
bbbbbbbbbb118.png
pxjkzvqomp51.png
mkmgcxaztt98.png
novarhxpbj44.png
mkmgcxaztt6.png
aayfryxljh123.png
eeeeeeeeee108.png
novarhxpbj47.png
pxjkzvqomp137.png
adohdulfwb30.png
oojxonbgow12.png
adohdulfwb9.png
dddddddddd115.png
pxjkzvqomp5.png
dddddddddd6.png
aaaaaaaaaa103.png
jzmzdispyo94.png
oojxonbgow70.png
dpevefkefv65.png
opvqd

In [15]:
gen_X_train=eyes_df[eyes_df['Name'].isin(X_train.values)]
gen_X_train=gen_X_train[['Name','Eyes']]
gen_X_train=shuffle(gen_X_train)
gen_X_train.reset_index(inplace=True, drop=True)
gen_X_train['Eyes'] = gen_X_train['Eyes'].astype(str)
gen_X_train

,Name,Eyes
0,cccccccccc25.png,0
1,aaaaaaaaaa0.png,1
2,pxjkzvqomp97.png,1
3,novarhxpbj85.png,1
4,adohdulfwb87.png,1
...,...,...
1659,jzmzdispyo112.png,1
1660,oojxonbgow44.png,1
1661,mkmgcxaztt138.png,1
1662,aayfryxljh44.png,1


In [16]:
# Have to increase our dataset... 2000 images leads to overfitting.
train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40,\
                                   width_shift_range = 0.2, height_shift_range = 0.2,\
                                   shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

In [17]:
# Extract names of images and associated labels from dataframe X_train
train_generator = train_datagen.flow_from_dataframe(dataframe=gen_X_train,\
                                                    directory="/content/Feature1_datacrop",\
                                                    x_col="Name",\
                                                    y_col="Eyes",\
                                                    batch_size = 20,\
                                                    class_mode = 'binary',\
                                                    target_size = (224, 224))

Found 1664 validated image filenames belonging to 2 classes.


In [18]:
# Prepare images.
def images_ready(all_imgs,folder_name):
    result=[]
    for i in all_imgs:
        # load an image from file
        get_file="/content/"+folder_name+str(i)
        image = load_img(get_file, target_size=(224, 224))
        # plt.imread(get_file)
        # load_img(get_file, target_size=(432, 288))
        # convert the image pixels to a numpy array
        image = img_to_array(image)
        # reshape data for the model
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        # prepare the image for the VGG model
        image = preprocess_input(image)
        result.append(image)
    return result

review_later=X_test
X_train=images_ready(X_train,"Feature1_datacrop/")
X_test=images_ready(X_test,"Feature1_datacrop/")

In [19]:
# Transfer learn VGG16 to detect eye blinks... binary classification.
# load model without classifier layers
model = VGG16(include_top=False, input_shape=(224, 224, 3))
# don't train upper layers
for layer in model.layers:
    layer.trainable = False
# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
output = Dense(1, activation='sigmoid')(class1)
# define new model
model = Model(inputs=model.inputs, outputs=output)
# summarize
model.summary()

58900480/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
______________________________________________________________

In [20]:
model.compile(loss='binary_crossentropy',optimizer=RMSprop(lr=0.0001),metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [21]:
regular_1=np.array(X_train)
good_shape_1=np.squeeze(regular_1)
print(regular_1.shape)
print(good_shape_1.shape)
regular_2=np.array(X_test)
good_shape_2=np.squeeze(regular_2)

(1664, 1, 224, 224, 3)
(1664, 224, 224, 3)


In [22]:
print(good_shape_1.shape)
print(y_train.shape)
# good_shape=good_shape[:10][:][:][:]
# y_train=y_train[:10][:]

(1664, 224, 224, 3)
(1664,)


In [23]:
sess= tf.compat.v1.Session(config=config)

In [24]:
checkpoint_path = "all_weights_eyeblink.ckpt"
checkpoint_dir = "/content/drive/MyDrive"+checkpoint_path

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [25]:
model.fit(good_shape_1, y_train, epochs=5, callbacks=[cp_callback])
# model.fit(train_generator, epochs=5, verbose=0)

Epoch 1/5
52/52 [==============================] - 22s 67ms/step - loss: 1.4093 - accuracy: 0.9213

Epoch 00001: saving model to all_weights_eyeblink.ckpt
Epoch 2/5
52/52 [==============================] - 4s 68ms/step - loss: 0.3893 - accuracy: 0.9591

Epoch 00002: saving model to all_weights_eyeblink.ckpt
Epoch 3/5
52/52 [==============================] - 4s 68ms/step - loss: 0.1545 - accuracy: 0.9766

Epoch 00003: saving model to all_weights_eyeblink.ckpt
Epoch 4/5
52/52 [==============================] - 4s 67ms/step - loss: 0.1845 - accuracy: 0.9808

Epoch 00004: saving model to all_weights_eyeblink.ckpt
Epoch 5/5
52/52 [==============================] - 3s 67ms/step - loss: 0.0713 - accuracy: 0.9892

Epoch 00005: saving model to all_weights_eyeblink.ckpt


In [26]:
print(good_shape_2.shape)
print(y_test.shape)

(416, 224, 224, 3)
(416,)


In [27]:
result_vgg16= model.predict(good_shape_2)
# (model.predict(good_shape_2) > 0.5).astype("int32")
# model.predict(good_shape_2)
print(result_vgg16)

[[9.99738991e-01]
 [0.00000000e+00]
 [1.00000000e+00]
 [1.00000000e+00]
 [1.00000000e+00]
 [3.96537296e-02]
 [5.60199540e-20]
 [9.49262555e-22]
 [1.00000000e+00]
 [1.00000000e+00]
 [4.88101414e-24]
 [1.00000000e+00]
 [1.00000000e+00]
 [1.00000000e+00]
 [1.51165175e-25]
 [4.96248007e-02]
 [1.00000000e+00]
 [1.00000000e+00]
 [1.00000000e+00]
 [1.00000000e+00]
 [9.96690869e-01]
 [1.00000000e+00]
 [1.00000000e+00]
 [1.00000000e+00]
 [1.00000000e+00]
 [8.03191592e-31]
 [9.64182854e-01]
 [1.00000000e+00]
 [0.00000000e+00]
 [9.99890327e-01]
 [1.00000000e+00]
 [2.64285342e-22]
 [9.99998689e-01]
 [1.60465474e-10]
 [1.00000000e+00]
 [1.00000000e+00]
 [1.00000000e+00]
 [0.00000000e+00]
 [5.56918844e-33]
 [0.00000000e+00]
 [1.00000000e+00]
 [1.00000000e+00]
 [1.00000000e+00]
 [2.13968627e-13]
 [1.00000000e+00]
 [8.17492858e-27]
 [8.69958401e-01]
 [9.99563515e-01]
 [1.00000000e+00]
 [1.00000000e+00]
 [1.00000000e+00]
 [7.67406060e-22]
 [1.00000000e+00]
 [2.15341791e-21]
 [3.88192101e-09]
 [1.000000

In [28]:
for i in range(0,len(result_vgg16)):
  if result_vgg16[i]>0.5:
    result_vgg16[i]=1
  else:
    result_vgg16[i]=0

In [29]:
print(np.array(y_test))

[1 0 1 1 1 1 0 0 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 0 1 1 1
 0 0 0 1 1 1 0 1 0 0 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 0 0 0 0 1 1 1 0 1 1 0 1
 1 1 1 0 0 0 1 1 0 1 1 1 1 0 0 1 0 1 0 0 0 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1
 0 0 1 0 1 0 1 0 1 0 1 1 0 0 1 1 1 1 1 0 0 1 0 1 0 0 1 0 1 1 1 0 1 0 1 1 1
 1 0 1 0 1 1 0 1 1 1 1 1 0 0 1 1 0 1 0 1 0 0 1 1 1 1 1 1 1 1 0 0 1 1 1 0 0
 1 1 0 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 0 1 0 1 1 0 1 0 1 1 1
 0 1 1 1 0 0 1 1 1 0 1 0 1 1 1 0 1 0 0 1 0 1 1 1 1 0 1 1 1 0 1 0 1 0 0 0 1
 1 1 1 0 0 1 1 0 0 1 1 1 1 1 0 1 1 0 1 0 0 1 0 0 1 1 0 1 1 1 1 0 1 1 1 0 0
 1 1 1 0 0 1 0 1 1 1 0 1 1 0 0 1 0 1 0 1 0 1 1 0 1 0 1 0 0 1 1 0 1 0 1 1 1
 0 0 1 1 1 1 0 1 1 1 1 1 1 0 0 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 0 1
 0 1 0 1 1 1 0 0 1 0 0 1 1 0 1 1 1 1 0 1 0 0 1 0 1 0 0 0 0 1 1 0 0 1 0 1 1
 1 0 1 1 1 1 1 1 1]


In [30]:
print("The model has an accuracy of:",accuracy_score(np.array(y_test), result_vgg16))

The model has an accuracy of: 0.9495192307692307


In [31]:
check_imgs=list()
for i in range(0,len(result_vgg16)):
  if result_vgg16[i]!=(np.array(y_test))[i]:
    check_imgs.append((np.array(review_later))[i])
print("The incorrectly predicted images were as follow:")
for i in range(0,len(check_imgs)):
  print(check_imgs[i])

The incorrectly predicted images were as follow:
pxjkzvqomp19.png
aaaaaaaaaa72.png
novarhxpbj47.png
aaaaaaaaaa103.png
adohdulfwb136.png
dpevefkefv2.png
aayfryxljh80.png
pxjkzvqomp52.png
pxjkzvqomp39.png
novarhxpbj23.png
opvqdabdap77.png
pxjkzvqomp36.png
aayfryxljh39.png
dpevefkefv19.png
aayfryxljh37.png
aayfryxljh118.png
dpevefkefv14.png
aayfryxljh13.png
opvqdabdap89.png
aaaaaaaaaa133.png
oojxonbgow73.png


In [32]:
# NOTE: Accuracy is a little different per notebook run.
# Although it's a 95% accuracy, the images that the model is getting incorrect are from the normal
# videos of people closing their eyes.
# 2091 total images.
# 600 images across 4 videos... each one person eyes closed (meditating, blind).
# 131 images from the regular videos of people blinking.
# 1360 images from regular talking videos.
# 20% of total images is testing set.
# 6% (131) of total images is from regular videos peoples eyes closed.
# Therefore our testing set has 0.06(0.20(2091))=25 images.
# We got about 20 images wrong from 25 which is 25% accuracy.
# Lets test this out with images from a video that has abnormally high blinking.
# 150 frames from a man trying to break fastest eye blink world record.
# If our model is good, then it will predict many more frames to have eyes closed. 

In [33]:
def more_tests(folder_dir, name_csv):
  extra_test = pd.read_csv('drive/MyDrive/'+name_csv, header=0)
  fast_blink_names=extra_test['Name']
  fast_blink_names=fast_blink_names.to_numpy()
  fast_blink_names=images_ready(fast_blink_names, folder_dir+"/")
  regular_3=np.array(fast_blink_names)
  good_shape_3=np.squeeze(regular_3)
  result_vgg16_2= model.predict(good_shape_3)
  count_1=0
  for i in range(0,len(result_vgg16_2)):
    if result_vgg16_2[i]>0.5:
      count_1+=1
  print("The model predicts",(count_1/150)*100,"% of the frames are eyes open.")
  print("The model predicts",100-((count_1/150)*100),"% of the frames are eyes closed.")

In [34]:
more_tests("Feature1_datatestcrop","Feature1_datatest.csv")

The model predicts 56.666666666666664 % of the frames are eyes open.
The model predicts 43.333333333333336 % of the frames are eyes closed.


In [35]:
# It is able to tell when there is a highly abnormal amount of blinking!
# The true value is more like 60% open, 40% closed, but that may not be an issue.
# The more frames the model thinks eyes closed, the more we can suspect abnormal blinking.
# If eyes open rate is near 100%, almost no blinking so that's also abnormal.
# Either way, reason to speculate a deepfake.

In [36]:
# Now test on video of person staring at the camera...
# 100% eyes open.

In [37]:
more_tests("Feature1_datatest2crop","Feature1_datatest2.csv")

The model predicts 99.33333333333333 % of the frames are eyes open.
The model predicts 0.6666666666666714 % of the frames are eyes closed.


In [38]:
# Improvements that can be made:
# Use some module to detect eye landmarks... crop the image to only get eyes...
# use those images as training data to the model.